In [1]:
from datasets import load_dataset
import pandas as pd
import random

# load the commonsense_qa trian set and convert to pandas
dataset = load_dataset("tau/commonsense_qa")
train = pd.DataFrame(dataset["train"])



/Users/chris/Desktop/paper-rep/learning-STaR/base/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train_size = len(train)

# # randomly choose 10 indices from the training set to use for few-shot prompting
# # # these 10 indices will stay the same throughout the whole STaR loop
# # random.seed(67)
# # random_indices = random.sample(range(train_size), 10)
# # exclude the randomly selected indices from the training set

# random_indices = [1225, 1902, 6690, 7661, 6804, 4372, 7036, 7991, 4555, 3338]

# print(random_indices)

### Build fewshot prompt example set

see fewshot_prompt.txt

### Test building prompt

In [4]:
# Randomly sample one question from the train dataset

# Read the few-shot prompt from the file
with open('data/prompts/fewshot_prompt.txt', 'r') as f:
    prompt = f.read()



random_idx = random.randint(0, len(train) - 1)
sample_question = train.iloc[random_idx]

# Extract question and format answer choices
question_text = sample_question['question']
choices_text = sample_question['choices']['text']

# Format answer choices as (a) choice1 (b) choice2 etc.
formatted_choices = ""
for i, choice in enumerate(choices_text):
    letter = chr(ord('a') + i)  # Convert 0,1,2,3,4 to a,b,c,d,e
    formatted_choices += f"({letter}) {choice}\n"

# Remove the trailing newline
formatted_choices = formatted_choices.strip()

# Replace placeholders in the prompt
prompt = prompt.replace("{{question}}", question_text)
test_prompt = prompt.replace("{{answer_choices}}", formatted_choices)


print(test_prompt)

### Example 1
Q: Sarah felt bad about their odds. Billy disagreed with her. What did he think about their odds?
Answer Choices:
(a) first rate
(b) adequate
(c) good
(d) propitious
(e) sufficient
A: The answer must be the opposite of Sarah's view since Billy disagreed. Sarah "felt bad," so Billy must think positively. 'Adequate' and 'sufficient' mean barely acceptable, not the opposite of bad. 'First rate' typically describes quality rather than odds. 'Good' is positive but generic. Only option D is a term specifically used to describe fortunate circumstances. Therefore, the answer is propitious (d).

### Example 2
Q: Where is an expressway unlikely to be found?
Answer Choices:
(a) ocean
(b) map
(c) canada
(d) large city
(e) country
A: The answer must be a place where expressways typically don't exist. A map can depict an expressway. Canada and large cities have expressways. An ocean cannot have roads at all, but the question asks "unlikely" not "impossible." Rural areas (the "country")

### import model and test prompt